In [1]:
import numpy as np
import pandas as pd

In [2]:
class NaiveBayesClassifier():
    def fit(self, X, y):
        ''' Calculate prior for each class '''
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)
        target_name = y.columns[0]
        
        self._means = np.zeros((n_classes, n_features-1), dtype=np.float64)
        self._vars = np.zeros((n_classes, n_features-1), dtype=np.float64)
        self._priors = np.zeros((n_classes), dtype=np.float64)

        for idx, c in enumerate(self._classes):
            X_c = X.query(f"{target_name} == {c}").drop(f"{target_name}", axis=1)
            self._means[idx, :] = X_c.mean(axis=0)
            self._vars[idx, :] = X_c.var(axis=0)
            self._priors[idx] = X_c.shape[0] / float(n_samples)

    def predict(self, X):
        ''' Calculate posterior for each class given inputs '''
        y_pred = [self._predict_one(x) for x in X.values] 
        return np.array(y_pred)
        
    def _predict_one(self, x):
        posteriors = [] # the posterior probabilities for each class according to this datapoint (x)

        # calculate the posteriors for each class on this datapoint
        for class_idx, c in enumerate(self._classes):
            prior = np.log(self._priors[class_idx])
            likelihoods = self._calculate_likelihoods_gaussian(class_idx, x)
            posterior = np.sum(np.log(likelihoods)) + prior
            posteriors.append(posterior)

        y_hat = np.argmax(posteriors)
        return self._classes[y_hat] # return the class with the highest posterior prob

    def _calculate_likelihoods_gaussian(self, class_idx, x):
        ''' compute the likelyhoods ( P(x_0|y), P(x_1|y), ...,  P(x_n|y)) using gaussian approximation '''
        mean = self._means[class_idx]
        var = self._vars[class_idx]
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(var) * np.sqrt(2 * np.pi)
        return numerator / denominator

The classic Iris classification dataset is a good example of the type of dataset that the Gaussian Naive Bayes algorithm works on. It's inputs are all numerical features.

In [3]:
import sklearn.datasets
features = ['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']
data = sklearn.datasets.load_iris(as_frame=True)
iris_df = data.frame

In [4]:
from sklearn.model_selection import train_test_split
X, y = iris_df.iloc[:,:-1], iris_df.iloc[:,-1:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

In [5]:
nb_model = NaiveBayesClassifier()
nb_model.fit(pd.concat([X_train, y_train], axis=1), y_train)

In [6]:
y_pred = nb_model.predict(X_test)

In [11]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[14  0  0]
 [ 0 12  3]
 [ 0  0 16]]
0.9333333333333333
